In [ ]:
# Instalación de librerías necesarias
!pip install --upgrade yt-dlp moviepy pydub speechrecognition transformers torch --quiet
!pip install git+https://github.com/openai/whisper.git
!apt update && apt install -y ffmpeg

# Autenticación en Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Crear la carpeta TFM en Google Drive
import os
output_dir = '/content/drive/MyDrive/TFM'
os.makedirs(output_dir, exist_ok=True)

# Importación de las librerías
import subprocess
import yt_dlp
from moviepy.editor import VideoFileClip, AudioFileClip
from pydub import AudioSegment
import whisper
import speech_recognition as sr
from transformers import pipeline
import time
import math

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-qye9aevw
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-qye9aevw
  Resolved https://github.com/openai/whisper.git to commit 173ff7dd1d9fb1c4fddea0d41d704cfefeb8908c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://

In [ ]:
# Definir el nombre del canal y las URLs de los videos
canal = "CHINOLO44"
video_urls = [
    'https://www.youtube.com/watch?v=EfJOqkwm-Pc',
    'https://www.youtube.com/watch?v=0F__iG2de6E',
    'https://www.youtube.com/watch?v=eGh1aOZiaj0'
]

In [ ]:
def descargar_videos(urls, canal):
    for idx, url in enumerate(urls, start=1):
        filename = f"{canal}_{idx}.mp4"
        filepath = os.path.join(output_dir, filename)
        ydl_opts = {
            'format': 'bestaudio+bestaudio',
            'outtmpl': filepath
        }
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            print(f"Descargando: {url}")
            ydl.download([url])
        print(f"Video descargado: {filename}")

def convertir_a_mp3_ffmpeg(video_path, output_name):
    audio_path = os.path.join(output_dir, f"{output_name}_ffmpeg.mp3")
    subprocess.run(['ffmpeg', '-i', video_path, audio_path])
    return audio_path

def convertir_a_mp3_pydub(video_path, output_name):
    audio = AudioSegment.from_file(video_path)
    audio_path = os.path.join(output_dir, f"{output_name}_pydub.mp3")
    audio.export(audio_path, format='mp3')
    return audio_path

def convertir_a_wav_moviepy(video_path, output_dir, output_name):
    wav_path = os.path.join(output_dir, f"{output_name}.wav")
    audio_clip = AudioFileClip(video_path)
    audio_clip.write_audiofile(wav_path, codec='pcm_s16le')
    audio_clip.close()
    return wav_path

def transcribir_whisper(audio_path):
    model = whisper.load_model("base")
    result = model.transcribe(audio_path)
    return result['text']

def transcribir_speech_recognition(audio_path):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio = recognizer.record(source)
    return recognizer.recognize_google(audio)

def transcribir_transformers(audio_path):
    transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-base")
    audio = AudioSegment.from_mp3(audio_path)
    chunk_length = 30 * 1000
    chunks = [audio[i:i + chunk_length] for i in range(0, len(audio), chunk_length)]

    transcription = ''
    for chunk in chunks:
        chunk.export("temp_chunk.wav", format="wav")
        with open("temp_chunk.wav", 'rb') as audio_file:
            transcription += transcriber(audio_file.read())['text'] + ' '
    return transcription.strip()

In [ ]:
print("Iniciando descarga de videos...")
start_time = time.perf_counter()
descargar_videos(video_urls, canal)
end_time = time.perf_counter()
print(f"Tiempo total de descarga de videos: {end_time - start_time:.2f} segundos\n")

Iniciando descarga de videos...
Descargando: https://www.youtube.com/watch?v=EfJOqkwm-Pc
[youtube] Extracting URL: https://www.youtube.com/watch?v=EfJOqkwm-Pc
[youtube] EfJOqkwm-Pc: Downloading webpage
[youtube] EfJOqkwm-Pc: Downloading ios player API JSON
[youtube] EfJOqkwm-Pc: Downloading mweb player API JSON
[youtube] EfJOqkwm-Pc: Downloading player 0ccfa671
[youtube] EfJOqkwm-Pc: Downloading m3u8 information
[info] EfJOqkwm-Pc: Downloading 1 format(s): 251
[download] Destination: /content/drive/MyDrive/TFM/CHINOLO44_1.mp4
[download] 100% of    9.42MiB in 00:00:02 at 3.58MiB/s   
Video descargado: CHINOLO44_1.mp4
Descargando: https://www.youtube.com/watch?v=0F__iG2de6E
[youtube] Extracting URL: https://www.youtube.com/watch?v=0F__iG2de6E
[youtube] 0F__iG2de6E: Downloading webpage
[youtube] 0F__iG2de6E: Downloading ios player API JSON
[youtube] 0F__iG2de6E: Downloading mweb player API JSON
[youtube] 0F__iG2de6E: Downloading m3u8 information
[info] 0F__iG2de6E: Downloading 1 format(s)

In [ ]:
for idx, _ in enumerate(video_urls, start=1):
    video_file = os.path.join(output_dir, f"{canal}_{idx}.mp4")
    output_name = f"{canal}_{idx}"

    print(f"\nConvirtiendo video {output_name} a MP3...")

    # MoviePy
    start_time = time.perf_counter()
    mp3_moviepy = convertir_a_wav_moviepy(video_file, output_dir, output_name)
    end_time = time.perf_counter()
    print(f"Tiempo de conversión con MoviePy: {end_time - start_time:.2f} segundos")

    # FFmpeg
    start_time = time.perf_counter()
    mp3_ffmpeg = convertir_a_mp3_ffmpeg(video_file, output_name)
    end_time = time.perf_counter()
    print(f"Tiempo de conversión con FFmpeg: {end_time - start_time:.2f} segundos")

    # Pydub
    start_time = time.perf_counter()
    mp3_pydub = convertir_a_mp3_pydub(video_file, output_name)
    end_time = time.perf_counter()
    print(f"Tiempo de conversión con Pydub: {end_time - start_time:.2f} segundos")


Convirtiendo video CHINOLO44_1 a MP3...
MoviePy - Writing audio in /content/drive/MyDrive/TFM/CHINOLO44_1.wav


MoviePy - Done.
Tiempo de conversión con MoviePy: 13.12 segundos
Tiempo de conversión con FFmpeg: 25.40 segundos
Tiempo de conversión con Pydub: 27.16 segundos

Convirtiendo video CHINOLO44_2 a MP3...
MoviePy - Writing audio in /content/drive/MyDrive/TFM/CHINOLO44_2.wav


MoviePy - Done.
Tiempo de conversión con MoviePy: 20.82 segundos
Tiempo de conversión con FFmpeg: 42.73 segundos
Tiempo de conversión con Pydub: 46.56 segundos

Convirtiendo video CHINOLO44_3 a MP3...
MoviePy - Writing audio in /content/drive/MyDrive/TFM/CHINOLO44_3.wav


MoviePy - Done.
Tiempo de conversión con MoviePy: 58.65 segundos
Tiempo de conversión con FFmpeg: 117.63 segundos
Tiempo de conversión con Pydub: 144.80 segundos


In [ ]:
mp3_ffmpeg = '/content/drive/MyDrive/TFM/CHINOLO44_1_ffmpeg.mp3'

mp3_ffmpeg_2 = '/content/drive/MyDrive/TFM/CHINOLO44_2_ffmpeg.mp3'

mp3_ffmpeg_3 = '/content/drive/MyDrive/TFM/CHINOLO44_2_ffmpeg.mp3'

In [ ]:
for idx, _ in enumerate(video_urls, start=1):
    video_file = os.path.join(output_dir, f"{canal}_{idx}.mp4")
    output_name = f"{canal}_{idx}"

    print("\nTranscribiendo archivos MP3...")

    # Whisper
    start_time = time.perf_counter()
    transcripcion_whisper = transcribir_whisper(mp3_ffmpeg)
    end_time = time.perf_counter()
    print(f"Tiempo de transcripción con Whisper: {end_time - start_time:.2f} segundos")
    print(f"Transcripción Whisper:\n{transcripcion_whisper}\n")

    # Transformers
    start_time = time.perf_counter()
    transcripcion_transformers = transcribir_transformers(mp3_ffmpeg)
    end_time = time.perf_counter()
    print(f"Tiempo de transcripción con Transformers: {end_time - start_time:.2f} segundos")
    print(f"Transcripción Transformers:\n{transcripcion_transformers}\n")

    # Whisper
    start_time = time.perf_counter()
    transcripcion_whisper = transcribir_whisper(mp3_ffmpeg_2)
    end_time = time.perf_counter()
    print(f"Tiempo de transcripción con Whisper: {end_time - start_time:.2f} segundos")
    print(f"Transcripción Whisper:\n{transcripcion_whisper}\n")

    # Transformers
    start_time = time.perf_counter()
    transcripcion_transformers = transcribir_transformers(mp3_ffmpeg_2)
    end_time = time.perf_counter()
    print(f"Tiempo de transcripción con Transformers: {end_time - start_time:.2f} segundos")
    print(f"Transcripción Transformers:\n{transcripcion_transformers}\n")

    # Whisper
    start_time = time.perf_counter()
    transcripcion_whisper = transcribir_whisper(mp3_ffmpeg_3)
    end_time = time.perf_counter()
    print(f"Tiempo de transcripción con Whisper: {end_time - start_time:.2f} segundos")
    print(f"Transcripción Whisper:\n{transcripcion_whisper}\n")

    # Transformers
    start_time = time.perf_counter()
    transcripcion_transformers = transcribir_transformers(mp3_ffmpeg_3)
    end_time = time.perf_counter()
    print(f"Tiempo de transcripción con Transformers: {end_time - start_time:.2f} segundos")
    print(f"Transcripción Transformers:\n{transcripcion_transformers}\n")


Transcribiendo archivos MP3...


  checkpoint = torch.load(fp, map_location=device)

  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



Tiempo de transcripción con Whisper: 198.40 segundos
Transcripción Whisper:
 que pasa chavales bienvenidos a chinolo bueno noticia triste no noticia triste me acabo de enterar y digo joder yo tengo que hacer un vídeo de Antonio no todo improvisación no tengo nada no tengo nada notado porque me acabo de enterar como ya os digo y la verdad me ha puesto hasta un poco triste no la noticia porque para los que llevamos siguiendo en los muchos años etcétera Antonio siempre ha sido un tío que estaba ahí en el sistema digamos no hasta o en la hasta o en la pomada y me ha puesto hasta triste no vamos a dedicar unas palabras ha improvisado un poco he pensado de cosas buenas y cosas malas no porque ya os digo que me ha puesto hasta un poco triste se van los mejores no entonces bueno lo primero que hay que yo el titular no el titular lo diría de ganar wols a una prejuvilación anticipada no o sea hace de un mes estaba diciendo que iba a ganar la lequí y va a ganar wols bueno al final la realidad es 

The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

...

Tiempo de transcripción con Transformers: 389.99 segundos
Transcripción Transformers:
Bien chavales de puta madre acababa de grabar el vídeo entero y me da cuenta de que tenía el sonido desactiva impresionante, no? Chino lo en su prínode sus normal me caú en la puta macho, me he grabado 20 minutos de vídeo 15 minutos y tenía el sonido apagado, tío. Así que nada voy a grabar lo otra, he visto lo visto, porque no me jodas. Bueno, os quería hablar un poco que ya he hablado, pero... Pero como no lo he grabado, pues lo vuelvo a grabar.  Ahora que después de retirarse.  pero os voy a hablar un poco del POKE para que veáis la fumada mental que lleva este tío porque yo he jugado bastante a POKE de manera profesional y sin improficional, he jugado no limit 25 de spinsan go, 7 mil spins de manera regular, ahí es donde lo dejé porque luego os contaré, no limit... bueno, he ganado torneos de 12k de 15k esto hace años, ¿por qué? porque hace años se podía ganar al POKE ¿por qué? porque tú te ponías 

  checkpoint = torch.load(fp, map_location=device)

  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



Tiempo de transcripción con Whisper: 403.37 segundos
Transcripción Whisper:
 Bien chábales de puta madre acababa de grabar el vídeo entero y me da cuenta de que tenía el sonido desactiva impresionante, no? Chino lo en su prínode sus normal me caúen la puta macho, me he grabado 20 minutos de vídeo 15 minutos y tenía el sonido apagado, tío. Así que nada voy a grabar otra vez, he visto lo visto, porque no me jodas. Bueno, os quería hablar un poco que ya he hablado, pero como no lo he grabado, pues lo vuelvo a grabar. Os quería hablar un poco que ha subido a Antonio este vídeo de, y ahora que no, después de retirarse. Entonces, me he atocado los cojones, me he atocado los cojones, porque bueno, el tío dice unas cosas hasta el minuto, cinco minutos, seis muy emotivos, muy bien, no? Bien, Antonio, en su sitio, tal, elegante, hasta que llega el momento de que le falta una idea, ¿no? Que es la realidad, o sea, que le falta una idea. Entonces, hoy os voy a hablar, este vídeo me apetece a hacerl

  warnings.warn(

  warnings.warn(

  warnings.warn(

  warnings.warn(

  warnings.warn(

  warnings.warn(

  warnings.warn(

  warnings.warn(

  warnings.warn(

  warnings.warn(

  warnings.warn(

  warnings.warn(

  warnings.warn(

  warnings.warn(

  warnings.warn(

  warnings.warn(

  warnings.warn(

  warnings.warn(

  warnings.warn(

  warnings.warn(

  warnings.warn(

  warnings.warn(

  warnings.warn(

  warnings.warn(

  warnings.warn(

  warnings.warn(

  warnings.warn(

  warnings.warn(

  warnings.warn(

  warnings.warn(

  warnings.warn(

  warnings.warn(

  warnings.warn(

  warnings.warn(

  warnings.warn(

  warnings.warn(

  warnings.warn(

  warnings.warn(



Tiempo de transcripción con Transformers: 392.13 segundos
Transcripción Transformers:
Bien chavales de puta madre acababa de grabar el vídeo entero y me da cuenta de que tenía el sonido desactiva impresionante, no? Chino lo en su prínode sus normal me caú en la puta macho, me he grabado 20 minutos de vídeo 15 minutos y tenía el sonido apagado, tío. Así que nada voy a grabar lo otra, he visto lo visto, porque no me jodas. Bueno, os quería hablar un poco que ya he hablado, pero... Pero como no lo he grabado, pues lo vuelvo a grabar.  Ahora que después de retirarse.  pero os voy a hablar un poco del POKE para que veáis la fumada mental que lleva este tío porque yo he jugado bastante a POKE de manera profesional y sin improficional, he jugado no limit 25 de spinsan go, 7 mil spins de manera regular, ahí es donde lo dejé porque luego os contaré, no limit... bueno, he ganado torneos de 12k de 15k esto hace años, ¿por qué? porque hace años se podía ganar al POKE ¿por qué? porque tú te ponías 